In [1]:
import pandas as pd

In [2]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')

In [3]:
bike_realtime_df.drop(columns="create_time",inplace=True)

In [4]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [5]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [6]:
# start_date = '2024-04-16'
# end_date = '2024-04-30'

# mrt_history_df = mrt_history_df[(mrt_history_df["date"] >= start_date) & (mrt_history_df["date"] <= end_date)]
# bike_realtime_df = bike_realtime_df[(bike_realtime_df["date"] >= start_date) & (bike_realtime_df["date"] <= end_date)]

In [7]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)


In [8]:
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [9]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [10]:
time_table_df.loc[0,"date"]

Timestamp('2020-01-10 00:00:00')

In [11]:
df.loc[0,"date"]

Timestamp('2024-04-16 00:00:00')

In [12]:
time_table_df.loc[0,"date"]

Timestamp('2020-01-10 00:00:00')

In [13]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")


In [14]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [15]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500101001,BR08,0.015941,科技大樓
1,500101002,BR08,0.053192,科技大樓
2,500101003,BR08,0.302305,科技大樓
3,500101004,BR08,0.296290,科技大樓
4,500101005,BR08,0.510676,科技大樓
...,...,...,...,...
1410,500119087,G07,0.645005,公館
1411,500119088,BR08,0.683471,科技大樓
1412,500119089,G07,0.884494,公館
1413,500119090,G07,0.719453,公館


In [16]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360,4,Wednesday
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360,4,Wednesday
...,...,...,...,...,...,...,...,...,...,...
1012257,500119091,2024-05-19,19,11.500000,6.500000,18,25.01816,121.54469,1,Sunday
1012258,500119091,2024-05-19,20,12.500000,5.500000,18,25.01816,121.54469,1,Sunday
1012259,500119091,2024-05-19,21,15.500000,2.500000,18,25.01816,121.54469,1,Sunday
1012260,500119091,2024-05-19,22,15.500000,2.500000,18,25.01816,121.54469,1,Sunday


In [17]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df.head()

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.5436,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.5436,4,Wednesday,BR08,0.015941,科技大樓


In [18]:
mrt_history_df = mrt_history_df.merge(time_table_df.loc[:,["date","day_of_week",]],
                                        on="date",
                                        how="left")

In [19]:
mrt_history_df_group_byweek_enter = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["enter_num"].mean()).reset_index(drop=False)
mrt_history_df_group_byweek_exit = pd.DataFrame(mrt_history_df.groupby(["mrt_station","day_of_week","hour"])["exit_num"].mean()).reset_index(drop=False)

In [20]:
mrt_history_df_group_byweek = mrt_history_df_group_byweek_enter.merge(
    mrt_history_df_group_byweek_exit,
    left_on = ["mrt_station","day_of_week","hour"],
    right_on = ["mrt_station","day_of_week","hour"],
    how = "left"
)
mrt_history_df_group_byweek

,mrt_station,day_of_week,hour,enter_num,exit_num
0,七張,1,0,34.777778,112.444444
1,七張,1,1,0.666667,9.111111
2,七張,1,5,0.222222,0.222222
3,七張,1,6,144.888889,85.555556
4,七張,1,7,328.333333,160.222222
...,...,...,...,...,...
17341,龍山寺,7,19,1731.888889,1695.111111
17342,龍山寺,7,20,1414.444444,1505.888889
17343,龍山寺,7,21,1342.555556,1470.444444
17344,龍山寺,7,22,893.222222,1232.111111


In [21]:
df = df.merge(
    mrt_history_df_group_byweek.loc[:,["day_of_week","hour","mrt_station","enter_num",	"exit_num"]],
    left_on=["day_of_week","hour",	"station_name"],
    right_on=["day_of_week","hour", "mrt_station"],
    how="left"
)

In [22]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name,mrt_station,enter_num,exit_num
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓,科技大樓,624.444444,402.222222
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓,科技大樓,198.555556,136.555556
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,82.500000,49.750000
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,科技大樓,0.500000,0.000000
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012257,500119091,2024-05-19,19,11.500000,6.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,624.222222,509.444444
1012258,500119091,2024-05-19,20,12.500000,5.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,550.111111,462.666667
1012259,500119091,2024-05-19,21,15.500000,2.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,429.666667,473.333333
1012260,500119091,2024-05-19,22,15.500000,2.500000,18,25.01816,121.54469,1,Sunday,BR08,0.890423,科技大樓,科技大樓,273.222222,351.000000


In [23]:
df["bike_rate"] = df["aval_bike_mean"]/df["total_space"]

In [24]:
df["enter_num"].fillna(0,inplace=True)
df["exit_num"].fillna(0,inplace=True)

In [25]:
df = df.loc[~df["hour"].isin([0,1,2,3,4,5,6]),]

In [26]:
bk_ids = list(df["bike_station_id"].unique())
df["total_visitors"] = df["enter_num"] + df["exit_num"]
df["ex_enter_diff"] =  df["exit_num"] - df["enter_num"] 

C:\Users\T14 Gen 3\AppData\Local\Temp\ipykernel_2748\3334949899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total_visitors"] = df["enter_num"] + df["exit_num"]
C:\Users\T14 Gen 3\AppData\Local\Temp\ipykernel_2748\3334949899.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ex_enter_diff"] =  df["exit_num"] - df["enter_num"]


In [27]:
import numpy as np

In [28]:
bk_ids = list(df["bike_station_id"].unique())
corr_df = pd.DataFrame(columns=["bike_station_id","corr_enter_bike","corr_exit_bike","corr_total_visitors","corr_ex_enter_diff","corr_enter_exit"])

In [29]:
for bk_id in bk_ids:
    tem = df.loc[df["bike_station_id"]==bk_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","total_visitors","ex_enter_diff","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "bike_station_id":[bk_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_total_visitors":cor_matrix.loc["total_visitors","bike_rate"],
        "corr_ex_enter_diff":cor_matrix.loc["ex_enter_diff","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)

C:\Users\T14 Gen 3\AppData\Local\Temp\ipykernel_2748\3527425555.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [30]:
corr_df.sort_values(["corr_exit_bike"],ascending=False)

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500107140,0.035692,0.536503,0.340229,0.426536,0.351243
0,500112082,-0.168740,0.509019,0.236936,0.467329,-0.034388
0,500107059,0.178131,0.496209,0.405194,0.263935,0.351243
0,500101156,0.390042,0.482516,0.480417,0.028599,0.620302
0,500101197,0.304011,0.482326,0.492056,0.099023,0.232821
...,...,...,...,...,...,...
0,500108179,NaN,NaN,NaN,NaN,NaN
0,500109065,NaN,NaN,NaN,NaN,0.797626
0,500111081,NaN,NaN,NaN,NaN,0.839165
0,500111085,NaN,NaN,NaN,NaN,0.726671


In [31]:
corr_df = pd.DataFrame(columns=["mrt_station","corr_enter_bike","corr_exit_bike","corr_enter_exit"])
mrt_ids = list(df["mrt_station"].unique())
for mrt_id in mrt_ids:
    tem = df.loc[df["mrt_station"]==mrt_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "mrt_station":[mrt_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


C:\Users\T14 Gen 3\AppData\Local\Temp\ipykernel_2748\1539274487.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [32]:
cor_matrix

,bike_rate,enter_num,exit_num
bike_rate,1.000000,-0.006720,0.006259
enter_num,-0.006720,1.000000,-0.164077
exit_num,0.006259,-0.164077,1.000000


In [33]:
corr_df.sort_values(["corr_enter_bike"],ascending=False)

,mrt_station,corr_enter_bike,corr_exit_bike,corr_enter_exit
0,中山,0.183028,0.314945,0.351243
0,台北車站,0.157244,0.254208,0.804252
0,三民高中,0.137701,-0.145006,-0.200410
0,忠孝復興,0.108551,0.108469,0.232821
0,忠孝敦化,0.096983,0.081619,0.183512
...,...,...,...,...
0,龍山寺,-0.147334,-0.078284,0.667477
0,南港軟體園區,-0.164764,0.078894,0.042047
0,新北投,-0.177576,-0.158318,0.624200
0,民權西路,-0.181754,0.138896,0.342230


In [34]:
df.to_csv("test.csv",index=False,encoding="utf-8-sig")